# Lab 10.1: CNN with MNIST Classifier

Edited By Steve Ive

Here, we are going to build simple CNN model for MNIST Classifying. To more understand about CNN, please refer before script "10.0 - About CNN".

Reference from

https://github.com/deeplearningzerotoall/PyTorch/blob/master/lab-10_1_mnist_cnn.ipynb

## Imports

In [12]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as datasets
import torchvision.transforms as transforms

In [13]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(1)

if device == 'cuda':
    torch.cuda.manual_seed(1)

## Load MNIST Data

In [14]:
mnist_train = datasets.MNIST(root = 'MNIST_data/',
                             download = True,
                             train = True,
                             transform = transforms.ToTensor())
mnist_test = datasets.MNIST(root = 'MNIST_data/',
                            download = True,
                            train = False,
                            transform = transforms.ToTensor())

In [15]:
data_loader = torch.utils.data.DataLoader(dataset = mnist_train, batch_size = batch_size, shuffle = True, drop_last = True)

## Set Hyperparameters

In [16]:
batch_size = 100
training_epochs = 20
learning_rate = 0.001

## Define Model

In [17]:
class MNIST_CNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Layer 1: ImageInput Shape=(?, 28, 28, 1)
        # Conv   : ImageOutput Shape=(?, 28, 28, 32)
        # Pool   : ImageOutput Shape=(?, 14, 14, 32)
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2)
        )
        # Layer 2: ImageInput Shape=(?, 14, 14, 32)
        # Conv   : ImageOutput Shape=(?, 14, 14, 64)
        # Pool   : ImageOutput Shape=(?, 7, 7, 64)
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size = 3, stride = 1, padding = 1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2),
        )
        # Final Fully Connected
        # 7 x 7 x 64 inputs => 10 outputs
        self.fc = nn.Linear(7 * 7 * 64, 10)
        init.xavier_uniform_(self.fc.weight)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        #print(out.size()) #torch.size([100, 64, 7, 7])
        out = out.view(out.size(0), -1) # Flatten for FC
        out = self.fc(out)
        return out    

In [18]:
model = MNIST_CNN().to(device)

In [19]:
optimizer = optim.Adam(model.parameters(), lr = learning_rate)

## Train

In [20]:
total_batch = len(data_loader)

print('Learning started')

for epoch in range(training_epochs):

    avg_cost = 0

    for X, Y in data_loader:
        # image is already size of (28 x 28), no reshape
        # Label is not one-hot encoded

        X = X.to(device)
        Y = Y.to(device)

        #prediction
        pred = model(X)

        #cost
        cost = F.cross_entropy(pred, Y).to(device)

        #Reduce the cost
        optimizer.zero_grad()
        cost.backward()
        optimizer.step()

        avg_cost += cost

    avg_cost = avg_cost / total_batch

    print('Epoch: {} / {}, cost: {:.6f}'.format(epoch + 1, training_epochs, avg_cost))
print('Learning Finished')

Learning started
Epoch: 1 / 20, cost: 0.218914
Epoch: 2 / 20, cost: 0.059701
Epoch: 3 / 20, cost: 0.044733
Epoch: 4 / 20, cost: 0.034433
Epoch: 5 / 20, cost: 0.029811
Epoch: 6 / 20, cost: 0.024734
Epoch: 7 / 20, cost: 0.020639
Epoch: 8 / 20, cost: 0.017615
Epoch: 9 / 20, cost: 0.015055
Epoch: 10 / 20, cost: 0.012490
Epoch: 11 / 20, cost: 0.010240
Epoch: 12 / 20, cost: 0.009451
Epoch: 13 / 20, cost: 0.007774
Epoch: 14 / 20, cost: 0.007936
Epoch: 15 / 20, cost: 0.005897
Epoch: 16 / 20, cost: 0.005412
Epoch: 17 / 20, cost: 0.003761
Epoch: 18 / 20, cost: 0.005102
Epoch: 19 / 20, cost: 0.004148
Epoch: 20 / 20, cost: 0.003553
Learning Finished


### Take a Moment!

Why we should ```.view(len(mnist_test), 1, 28 ,28)``` below despite of we didn't do that upon?

=> since ```data_loader``` makes mini batches as size of size([100, 1, 28, 28]), we don't have to do that manually, but we don't use ```data_loader``` below.

In [21]:
#Test model and check accuracy
with torch.no_grad():
    #print(mnist_test.data.shape) => torch.Size([10000, 28, 28])
    X_test = mnist_test.data.view(len(mnist_test), 1, 28, 28).float().to(device)
    Y_test = mnist_test.targets.to(device)

    #prediction
    pred = model(X_test)
    correct_prediction = (torch.argmax(pred, 1) == Y_test)
    accuracy = correct_prediction.float().mean()

    print('Accuracy: ', accuracy.item())

Accuracy:  0.9829999804496765
